## Web Scraping

In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Função para realização do web scraping
def search_thesis (programs):
  columns = ['program', 'day', 'month', 'year', 'title', 'author', 'key_words', 'abstract', 'resume']
  df = pd.DataFrame(columns=columns)

  url_general = 'http://repositorio.ufrn.br:8080/jspui/handle/123456789/'

  # Percorre os subprogramas dos programas
  for key, value in programs.items():

    url_program = url_general + value
    response = requests.get(url_program)
    content = response.content
    parser = BeautifulSoup(content, 'html.parser')

    # Total de teses/dissertações
    text_range = parser.select(".browse_range")[0].getText()
    num_range = int(text_range.strip().split(" ")[-1])
    num_offsets = int(num_range / 20)

    # Percorre a listagens das teses/dissertações do programa
    for offset in range(0, num_offsets+1):
      # Usado apenas para que seja definido o cookie com o idioma PT-BR
      url_cookie_pt_br = url_general + value + '?locale=pt_BR'
      reponse_cookie_pr_br = requests.get(url_cookie_pt_br)

      url_program_pages = url_general + value + '?offset=' + str(offset*20);

      response_program_pages = requests.get(url_program_pages)
      content_program_pages = response_program_pages.content
      parser_program_pages = BeautifulSoup(content_program_pages, 'html.parser')

      table = parser_program_pages.select(".table")

      if len(table) < 1:
        continue

      table = table[0]
      table_rows = table.find_all("tr")

      # Percorre os dados da tabela das teses/dissertações
      for cel in table_rows[1:]:

        date = cel.select("td")[0].getText()
        date = date.split('-')

        if len(date) == 3:
          day = date[0]
          month = date[1]
          year = date[2]
        else:
          day = None
          month = date[0]
          year = date[1]
        
        # Pode ser removido '&#64257' das strings também, se atrapalhar a analise dos dados;
        title = cel.select("td")[1].getText().replace('&#65279;', '')
        tese_link = 'http://repositorio.ufrn.br:8080' + cel.select("a")[0].get("href") + '?locale=pt_BR'
        author = cel.select("td")[2].getText().replace('&#65279;', '')

        response_tese = requests.get(tese_link)
        content_tese = response_tese.content
        parser_tese = BeautifulSoup(content_tese, 'html.parser')

        table_tese = parser_tese.select(".table")[0]
        table_rows_tese = table_tese.find_all("tr")
        
        infos = get_info(table_rows_tese)

        tese_df = pd.DataFrame([[key, day, month, year, title, author, infos[0], infos[1], infos[2]]], columns=columns)

        df = df.append(tese_df, ignore_index=True)
      
  return df

In [0]:
# Função que pega as palavras-chave, abstract e resumo da tabela na página de uma tese/dissertação especifica
def get_info (table_rows_thesis):
  key_words = None
  abstract = None
  resume = None
  
  for row in table_rows_thesis:    
    item = row.select("td")[0].getText().strip()

    if item == 'Palavras-chave:':
      key_words = row.select("td")[1].getText().strip('&#65279;').replace('&#65279;', '')
    elif item == 'Abstract:':
      abstract = row.select("td")[1].getText().strip('&#65279;').replace('&#65279;', '')
    elif item == 'Resumo:':
      resume = row.select("td")[1].getText().strip('&#65279;').replace('&#65279;', '')
  
  return [key_words, abstract, resume]

In [0]:
# Usado para calcular o tempo de execução
import time
start = time.time()

# Definição do código dos programas
ppgeec = {'doutorado':'12009', 'mestrado':'12008'}
ppged = {'doutorado':'12002', 'mestrado':'12001'}
ppgcsa = {'doutorado':'11917', 'mestrado':'11916'}

ppgcsa_df = search_thesis(ppgcsa)
ppgeec_df = search_thesis(ppgeec)
ppged_df = search_thesis(ppged)

# Usado para calcular o tempo de execução
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

00:27:10.36


## Save Datasets

In [0]:
# Datasets do PPGEEC
print(ppgeec_df.tail())
ppgeec_df.to_csv('ppgeec.csv')

ppgeec_mestrado_df = ppgeec_df[ppgeec_df.program == 'mestrado'].reset_index(drop=True)
ppgeec_mestrado_df.to_csv('ppgeec_mestrado.csv')

ppgeec_doutorado_df = ppgeec_df[ppgeec_df.program == 'doutorado'].reset_index(drop=True)
ppgeec_doutorado_df.to_csv('ppgeec_doutorado.csv')

      program  ...                                             resume
664  mestrado  ...  Este trabalho desenvolveu rotinas de software,...
665  mestrado  ...  Neste trabalho, a robustez e o desempenho do c...
666  mestrado  ...  Este trabalho se propõe a realizar uma análise...
667  mestrado  ...  No presente trabalho, aplicamos técnicas de in...
668  mestrado  ...  Esta dissertação descreve a utilização de nova...

[5 rows x 9 columns]


In [0]:
# Datasets do PPGCSA
print(ppgcsa_df.tail())
ppgcsa_df.to_csv('ppgcsa.csv')

ppgcsa_mestrado_df = ppgcsa_df[ppgcsa_df.program == 'mestrado'].reset_index(drop=True)
ppgcsa_mestrado_df.to_csv('ppgcsa_mestrado.csv')

ppgcsa_doutorado_df = ppgcsa_df[ppgcsa_df.program == 'doutorado'].reset_index(drop=True)
ppgcsa_doutorado_df.to_csv('ppgcsa_doutorado.csv')

      program  ...                                             resume
464  mestrado  ...  O objetivo deste estudo de caráter descritivo ...
465  mestrado  ...  Os índices de prevalência da obesidade crescem...
466  mestrado  ...  A violência institucional pode incluir desde a...
467  mestrado  ...  A elaboração de perfis de características que ...
468  mestrado  ...  A obesidade é considerada, atualmente, problem...

[5 rows x 9 columns]


In [0]:
# Datasets do PPGED
print(ppged_df.tail())
ppged_df.to_csv('ppged.csv')

ppged_mestrado_df = ppged_df[ppged_df.program == 'mestrado'].reset_index(drop=True)
ppged_mestrado_df.to_csv('ppged_mestrado.csv')

ppged_doutorado_df = ppged_df[ppged_df.program == 'doutorado'].reset_index(drop=True)
ppged_doutorado_df.to_csv('ppged_doutorado.csv')

      program  ...                                             resume
836  mestrado  ...  O objeto de estudo desta dissertação consiste ...
837  mestrado  ...  Esta pesquisa está centrada nas raízes da prop...
838  mestrado  ...  O presente trabalho tem como objetivo reconsti...
839  mestrado  ...  Tendo como objeto de estudo a Educação Física ...
840  mestrado  ...  Os dados mais recentes do Prova Brasil (BRASIL...

[5 rows x 9 columns]


## Basic Analysis

In [0]:
ppgeec_df.tail()

,program,day,month,year,title,author,key_words,abstract,resume
435,mestrado,15,Feb,2007,Software inteligente embarcado aplicado à corr...,"Cagni Júnior, Elói",None,"This study developed software rotines, in a sy...",None
436,mestrado,19,Oct,2007,"Avaliação de robustez, desempenho e aplicação ...","Santos, Marcelo Brandão dos",None,"In this work, the variable structure adaptive ...",None
437,mestrado,13,Nov,2007,Um sistema para estimação da vazão de gás de p...,"Silva, Sérgio José Gonçalves e",None,This work intends to analyze the behavior of t...,None
438,mestrado,23,Nov,2007,Técnicas de inteligência artificial para a ger...,"Araújo Júnior, José Medeiros de",None,Artificial Intelligence techniques are applied...,None
439,mestrado,31,Jul,2006,Utilização de Novas Tecnologias de Telecomunic...,"Jesus Júnior, Salvador Fernandes de",None,This dissertation describes the use of new Tec...,None


In [0]:
ppgeec_df[ppgeec_df.author == 'Avelino, Álvaro Medeiros']

,program,day,month,year,title,author,key_words,abstract,resume
34,doutorado,9,Jun,2017,LP-P2IP: uma versão de baixo consumo da arquit...,"Avelino, Álvaro Medeiros",FPGA;Reconfiguração parcial;Processamento de v...,None,Este trabalho utiliza como base a arquitetura ...


In [0]:
ppgeec_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669 entries, 0 to 668
Data columns (total 9 columns):
program      669 non-null object
day          667 non-null object
month        669 non-null object
year         669 non-null object
title        669 non-null object
author       669 non-null object
key_words    669 non-null object
abstract     627 non-null object
resume       669 non-null object
dtypes: object(9)
memory usage: 47.1+ KB


In [0]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(ppgeec_df)

In [0]:
years = ppgeec_df.year.unique()
years = [int(x) for x in years]
years.sort()

print(years)

[1985, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]


In [0]:
doutorado_df = ppgeec_df[ppgeec_df.program == 'doutorado'].reset_index(drop=True)
doutorado_df.head()

,program,day,month,year,title,author,key_words,abstract,resume
0,doutorado,17,Dez,2018,"N-Learning, uma abordagem para ensino e aprend...","Costa, Luís Feliphe Silva",Sistemas multirrobôs;Robôtica baseada em compo...,"We propose the N-Learning paradigm, which allo...","Propomos o paradigma N-Learning, que propicia ..."
1,doutorado,20,Dez,2018,SMART: Sistema de Monitoramento e Avaliação do...,"Paiva, Jailton Carlos de",Telessaúde;Avaliação de Programa;Interoperabil...,The Telehealth Program was established by the ...,O Programa Telessaúde foi instituído pelo Mini...
2,doutorado,21,Dez,2018,Estimação da taxa de consumo de oxigênio com f...,"Silva, Francisco Jadilson dos Santos",Taxa de consumo de oxigênio;Oxigênio dissolvid...,The oxygen uptake rate (OUR) is a key variable...,A taxa de consumo de oxigênio (TCO) é uma vari...
3,doutorado,28,Dez,2018,Antenas compactas e miniaturizadas para sistem...,"Souza, Eduardo Amorim Martins de",Antena planar de microfita;Miniaturização de a...,"Over the last few years, wireless communicatio...","Ao longo dos últimos anos, os sistemas de comu..."
4,doutorado,23,Nov,2018,Análise e projeto de um Conversor A/D Sigma De...,"Soares, Antonio Wallace Antunes",Conversores analógicos digitais;Sigma delta in...,Several applications in the instrumentation fi...,Diversas aplicações na área de instrumentação ...


In [0]:
mestrado_df = ppgeec_df[ppgeec_df.program == 'mestrado'].reset_index(drop=True)
mestrado_df.head()

,program,day,month,year,title,author,key_words,abstract,resume
0,mestrado,10,Jan,2014,Estudo da atenuação da onda eletromagnética e...,"Soffiatti, André",Antena de Microfita;Multicamadas Elétricas;Aco...,This work aims at investigating the behavior o...,Este trabalho tem como objetivo principal inve...
1,mestrado,None,Jul,2009,Firewall de aplicação para redes industriais,"Batista Júnior, Aguinaldo Bezerra",Proteção de infra-estrutura crítica;Firewall d...,The utilization of firewalls is a widely accep...,A utilização de firewalls é uma abordagem ampl...
2,mestrado,None,Fev,2003,Controle adaptativo indireto por modelo de ref...,"Oliveira, Josenalde Barbosa de",Controlador adaptativo;Motor de induçao trifásico,An indirect approach to the variable structure...,O objetivo deste trabalho é desenvolver um con...
3,mestrado,28,Jan,2019,Wavelet-based protection assessment of the dou...,"Marques, Max Rodrigues",Gerador de Indução Duplamente Alimentado;Prote...,With the globalization and the rapidly increas...,Com o aumento da globalização e da demanda ene...
4,mestrado,25,Jan,2019,Data-Driven approach to parametric configurati...,"Nunes, Yuri Thomas Pinheiro",Alarm configuration;Industrial alarms;Alarm tu...,As plantas industriais são compostas por proce...,Industrial plants are composed of processes th...


In [0]:
# Anos que contém dados
doutorado_years = doutorado_df.year.unique()
doutorado_years = [int(x) for x in doutorado_years]
doutorado_years.sort()

print('Doutorado:', doutorado_years)

mestrado_years = mestrado_df.year.unique()
mestrado_years = [int(x) for x in mestrado_years]
mestrado_years.sort()

print('Mestrado:', mestrado_years)

Doutorado: [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
Mestrado: [1985, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
